In [2]:

import os
os.environ["OMP_NUM_THREADS"] = "1"

import pandas as pd
from sklearn.cluster import KMeans
import folium
from folium.plugins import MarkerCluster
from flask import Flask, render_template_string, jsonify
from datetime import datetime
import threading
import time
import requests
from io import StringIO
import re

# =====================================================
# CONFIGURACIÓN
# =====================================================
CSV_URL = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTKaFPt5hGtEskfNrHmQbZwXnkVYE5hvVLqUMmP_gIEnX_0wCXF-DOFB8RYRZw80aJhqjtcbcp-kKYC/pub?gid=248747926&single=true&output=csv"
INTERVALO_REFRESH = 30
PUERTO = 5000

app = Flask(__name__)

# Variables globales
mapa_html = ""
ultima_actualizacion = ""
total_respuestas = 0
total_seguras = 0
total_moderadas = 0
total_peligrosas = 0

# =====================================================
# FUNCIÓN: BUSCAR COLUMNA DE FORMA FLEXIBLE
# =====================================================
def buscar_columna(df, palabras_clave):
    """
    Busca una columna usando palabras clave de forma flexible
    Ignora mayúsculas, espacios extra y caracteres especiales
    """
    for col in df.columns:
        col_limpia = col.strip().lower()
        col_limpia = re.sub(r'\s+', ' ', col_limpia)  # Normalizar espacios
        
        for palabra in palabras_clave:
            if palabra.lower() in col_limpia:
                return col
    return None

# =====================================================
# FUNCIÓN: LEER DATOS DESDE GOOGLE FORMS
# =====================================================
def leer_datos_google_forms():
    """Lee datos en tiempo real desde Google Sheets"""
    try:
        print("[🔄] Consultando Google Sheets...")
        
        response = requests.get(CSV_URL, timeout=10)
        response.raise_for_status()
        
        csv_data = StringIO(response.text)
        df = pd.read_csv(csv_data)
        
        if len(df) == 0:
            print("[!] No hay datos en el formulario todavía")
            return None
        
        print(f"[✓] Leídos {len(df)} registros")
        return df
        
    except Exception as e:
        print(f"[✗] Error: {e}")
        return None

# =====================================================
# FUNCIÓN: PREPARAR DATOS (VERSIÓN ROBUSTA)
# =====================================================
def preparar_datos(df):
    """Prepara datos usando búsqueda flexible de columnas"""
    
    print("\n📋 Columnas detectadas:")
    for i, col in enumerate(df.columns, 1):
        print(f"   {i}. '{col}'")
    
    # Limpiar nombres de columnas (quitar espacios extra)
    df.columns = df.columns.str.strip()
    
    # Buscar cada columna necesaria de forma flexible
    print("\n🔍 Buscando columnas...")
    
    mapa_columnas = {}
    
    # Buscar Latitud
    col_lat = buscar_columna(df, ["latitud", "lat"])
    if col_lat:
        mapa_columnas[col_lat] = "latitud"
        print(f"   ✓ Latitud encontrada: '{col_lat}'")
    else:
        print(f"   ✗ Latitud NO encontrada")
    
    # Buscar Longitud
    col_lon = buscar_columna(df, ["longitud", "long", "lon"])
    if col_lon:
        mapa_columnas[col_lon] = "longitud"
        print(f"   ✓ Longitud encontrada: '{col_lon}'")
    else:
        print(f"   ✗ Longitud NO encontrada")
    
    # Buscar Iluminación
    col_ilum = buscar_columna(df, ["iluminación", "iluminacion", "luz"])
    if col_ilum:
        mapa_columnas[col_ilum] = "iluminacion"
        print(f"   ✓ Iluminación encontrada: '{col_ilum}'")
    else:
        print(f"   ✗ Iluminación NO encontrada")
    
    # Buscar Comercios
    col_com = buscar_columna(df, ["comercio", "tienda", "negocio"])
    if col_com:
        mapa_columnas[col_com] = "comercios"
        print(f"   ✓ Comercios encontrada: '{col_com}'")
    else:
        print(f"   ✗ Comercios NO encontrada")
    
    # Buscar Reportes
    col_rep = buscar_columna(df, ["reporte", "incidente", "crimen"])
    if col_rep:
        mapa_columnas[col_rep] = "reportes"
        print(f"   ✓ Reportes encontrada: '{col_rep}'")
    else:
        print(f"   ✗ Reportes NO encontrada")
    
    # Buscar Flujo
    col_flu = buscar_columna(df, ["flujo", "persona", "gente", "tráfico"])
    if col_flu:
        mapa_columnas[col_flu] = "flujo"
        print(f"   ✓ Flujo encontrada: '{col_flu}'")
    else:
        print(f"   ✗ Flujo NO encontrada")
    
    # Buscar Distancia Policía
    col_pol = buscar_columna(df, ["distancia", "policía", "policia"])
    if col_pol:
        mapa_columnas[col_pol] = "distancia_policia"
        print(f"   ✓ Distancia policía encontrada: '{col_pol}'")
    else:
        print(f"   ✗ Distancia policía NO encontrada")
    
    # Renombrar columnas
    df = df.rename(columns=mapa_columnas)
    
    # Convertir comas a puntos en todas las columnas
    print("\n🔧 Normalizando números...")
    for col in df.columns:
        if df[col].dtype == 'object':
            try:
                # Reemplazar comas por puntos
                df[col] = df[col].astype(str).str.replace(',', '.', regex=False)
                # Quitar espacios
                df[col] = df[col].str.strip()
            except:
                pass
    
    print(f"\n📊 Columnas después de mapeo: {df.columns.tolist()}")
    
    return df

# =====================================================
# FUNCIÓN: GENERAR MAPA
# =====================================================
def generar_mapa():
    """Genera el mapa con clustering"""
    global mapa_html, ultima_actualizacion, total_respuestas
    global total_seguras, total_moderadas, total_peligrosas
    
    try:
        df = leer_datos_google_forms()
        
        if df is None or len(df) == 0:
            return False
        
        df = preparar_datos(df)
        
        # Verificar que tenemos las columnas necesarias
        required = ["latitud", "longitud", "iluminacion", "comercios", 
                   "reportes", "flujo", "distancia_policia"]
        
        columnas_actuales = df.columns.tolist()
        faltantes = [col for col in required if col not in columnas_actuales]
        
        if faltantes:
            print(f"\n⚠️  COLUMNAS FALTANTES: {faltantes}")
            print(f"⚠️  Columnas disponibles: {columnas_actuales}")
            return False
        
        # Convertir a numérico
        features = ["iluminacion", "comercios", "reportes", "flujo", "distancia_policia"]
        df[features] = df[features].apply(pd.to_numeric, errors="coerce")
        df = df.dropna(subset=features + ["latitud", "longitud"])
        
        if len(df) == 0:
            print("[!] No hay datos válidos después de limpiar")
            return False
        
        total_respuestas = len(df)
        
        # K-Means (solo si hay suficientes datos)
        if len(df) >= 3:
            # Clustering normal con 3 grupos
            kmeans = KMeans(n_clusters=3, random_state=0)
            df["cluster"] = kmeans.fit_predict(df[features])
            
            colores = {0: "green", 1: "orange", 2: "red"}
            df["color"] = df["cluster"].map(colores)
            
            # Estadísticas
            conteo = df["cluster"].value_counts().to_dict()
            total_seguras = conteo.get(0, 0)
            total_moderadas = conteo.get(1, 0)
            total_peligrosas = conteo.get(2, 0)
            
            print(f"[✓] Clustering aplicado con {len(df)} registros")
            
        elif len(df) == 2:
            # Con 2 respuestas: dividir en 2 grupos
            kmeans = KMeans(n_clusters=2, random_state=0)
            df["cluster"] = kmeans.fit_predict(df[features])
            
            # Mapear a colores (0=verde, 1=rojo)
            colores = {0: "green", 1: "red"}
            df["color"] = df["cluster"].map(colores)
            
            conteo = df["cluster"].value_counts().to_dict()
            total_seguras = conteo.get(0, 0)
            total_moderadas = 0
            total_peligrosas = conteo.get(1, 0)
            
            print(f"[⚠️] Solo 2 registros - usando 2 clusters temporalmente")
            
        else:
            # Con 1 respuesta: todo azul (neutral)
            df["cluster"] = 0
            df["color"] = "blue"
            
            total_seguras = 0
            total_moderadas = 0
            total_peligrosas = 0
            
            print(f"[⚠️] Solo 1 registro - sin clustering (necesitas al menos 3 respuestas)")
        
        # Crear mapa
        centro = [df["latitud"].mean(), df["longitud"].mean()]
        mapa = folium.Map(location=centro, zoom_start=14)
        marker_cluster = MarkerCluster().add_to(mapa)
        
        for _, row in df.iterrows():
            # Determinar nombre según cluster
            if len(df) == 1:
                nombre = "📍 Ubicación"
                color_nombre = "blue"
            elif row["cluster"] == 0:
                nombre = "🟢 Zona Segura"
                color_nombre = row["color"]
            elif row["cluster"] == 1:
                if len(df) == 2:
                    nombre = "🔴 Zona de Riesgo"
                else:
                    nombre = "🟠 Riesgo Moderado"
                color_nombre = row["color"]
            else:
                nombre = "🔴 Zona de Riesgo"
                color_nombre = row["color"]
            
            popup_html = f"""
            <div style="font-family: Arial; min-width: 220px;">
                <h4 style="margin: 5px 0; color: {color_nombre};">{nombre}</h4>
                <hr style="margin: 5px 0;">
                <b>💡 Iluminación:</b> {row['iluminacion']}<br>
                <b>🏪 Comercios:</b> {int(row['comercios'])}<br>
                <b>⚠️ Reportes:</b> {int(row['reportes'])}<br>
                <b>👥 Flujo:</b> {int(row['flujo'])}/5<br>
                <b>🚓 Dist. Policía:</b> {int(row['distancia_policia'])}m<br>
                <hr style="margin: 5px 0;">
                <small>📍 {row['latitud']:.6f}, {row['longitud']:.6f}</small>
                {'<br><small style="color: orange;">⚠️ Agrega más respuestas para clustering</small>' if len(df) < 3 else ''}
            </div>
            """
            
            folium.CircleMarker(
                location=[row["latitud"], row["longitud"]],
                radius=40,
                color=row["color"],
                fill=True,
                fill_color=row["color"],
                fill_opacity=0.7,
                popup=folium.Popup(popup_html, max_width=300)
            ).add_to(marker_cluster)
        
        # Guardar mapa como archivo temporal
        mapa.save('mapa_temp.html')
        
        # Leer el archivo
        with open('mapa_temp.html', 'r', encoding='utf-8') as f:
            mapa_html = f.read()
        
        ultima_actualizacion = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
        
        print(f"[✅] Mapa actualizado: {total_respuestas} total | 🟢 {total_seguras} | 🟠 {total_moderadas} | 🔴 {total_peligrosas}")
        return True
        
    except Exception as e:
        print(f"[✗] Error: {e}")
        import traceback
        traceback.print_exc()
        return False

# =====================================================
# ACTUALIZACIÓN AUTOMÁTICA
# =====================================================
def actualizar_periodicamente():
    while True:
        time.sleep(INTERVALO_REFRESH)
        generar_mapa()

# =====================================================
# INTERFAZ WEB
# =====================================================
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>🗺️ Mapa de Seguridad en Tiempo Real</title>
    <meta charset="utf-8">
    <meta http-equiv="refresh" content="{{ intervalo }}">
    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        body {
            font-family: Arial, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
        }
        .header {
            background: rgba(255, 255, 255, 0.98);
            padding: 25px;
            box-shadow: 0 4px 20px rgba(0,0,0,0.15);
        }
        .header h1 {
            color: #667eea;
            text-align: center;
            margin-bottom: 20px;
            font-size: 28px;
        }
        .stats {
            display: flex;
            justify-content: center;
            gap: 15px;
            flex-wrap: wrap;
            margin-bottom: 15px;
        }
        .stat-card {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 15px 25px;
            border-radius: 10px;
            box-shadow: 0 4px 15px rgba(0,0,0,0.2);
            min-width: 150px;
            text-align: center;
        }
        .stat-card h3 {
            font-size: 12px;
            font-weight: normal;
            opacity: 0.9;
            margin-bottom: 5px;
        }
        .stat-card p {
            font-size: 22px;
            font-weight: bold;
        }
        .pulse { animation: pulse 2s infinite; }
        @keyframes pulse { 0%, 100% { opacity: 1; } 50% { opacity: 0.7; } }
        .clusters {
            display: flex;
            gap: 10px;
            justify-content: center;
            flex-wrap: wrap;
        }
        .cluster-badge {
            display: flex;
            align-items: center;
            gap: 6px;
            background: white;
            color: #333;
            padding: 8px 15px;
            border-radius: 20px;
            font-size: 13px;
            font-weight: 600;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
        }
        .cluster-dot { width: 12px; height: 12px; border-radius: 50%; }
        #mapa-container { height: calc(100vh - 220px); padding: 20px; }
        #mapa-frame {
            width: 100%;
            height: 100%;
            border: none;
            border-radius: 15px;
            box-shadow: 0 10px 30px rgba(0,0,0,0.3);
            background: white;
        }
        .legend {
            position: fixed;
            bottom: 25px;
            right: 25px;
            background: rgba(255, 255, 255, 0.98);
            padding: 18px;
            border-radius: 12px;
            box-shadow: 0 6px 20px rgba(0,0,0,0.25);
        }
        .legend h4 { color: #667eea; margin-bottom: 12px; }
        .legend-item {
            display: flex;
            align-items: center;
            margin: 8px 0;
        }
        .legend-color {
            width: 18px;
            height: 18px;
            border-radius: 50%;
            margin-right: 10px;
        }
    </style>
</head>
<body>
    <div class="header">
        <h1>🗺️ Sistema de Monitoreo de Seguridad Urbana</h1>
        <div class="stats">
            <div class="stat-card">
                <h3>📝 RESPUESTAS</h3>
                <p>{{ total }}</p>
            </div>
            <div class="stat-card">
                <h3>🕐 ACTUALIZADO</h3>
                <p style="font-size: 16px;">{{ ultima }}</p>
            </div>
            <div class="stat-card pulse">
                <h3>⚡ AUTO-REFRESH</h3>
                <p style="font-size: 18px;">{{ intervalo }}s</p>
            </div>
            <div class="stat-card">
                <h3>🔗 FUENTE</h3>
                <p style="font-size: 16px;">Google Forms</p>
            </div>
        </div>
        <div class="clusters">
            <div class="cluster-badge">
                <span class="cluster-dot" style="background: green;"></span>
                <span>Seguras: {{ seguras }}</span>
            </div>
            <div class="cluster-badge">
                <span class="cluster-dot" style="background: orange;"></span>
                <span>Moderadas: {{ moderadas }}</span>
            </div>
            <div class="cluster-badge">
                <span class="cluster-dot" style="background: red;"></span>
                <span>Peligrosas: {{ peligrosas }}</span>
            </div>
        </div>
    </div>
    <div id="mapa-container">
        <iframe id="mapa-frame" src="/mapa"></iframe>
    </div>
    <div class="legend">
        <h4>📊 Clasificación</h4>
        <div class="legend-item">
            <div class="legend-color" style="background: green;"></div>
            <span><b>Zona Segura</b></span>
        </div>
        <div class="legend-item">
            <div class="legend-color" style="background: orange;"></div>
            <span><b>Riesgo Moderado</b></span>
        </div>
        <div class="legend-item">
            <div class="legend-color" style="background: red;"></div>
            <span><b>Zona de Riesgo</b></span>
        </div>
    </div>
</body>
</html>
"""

@app.route('/')
def index():
    return render_template_string(
        HTML_TEMPLATE,
        ultima=ultima_actualizacion,
        total=total_respuestas,
        intervalo=INTERVALO_REFRESH,
        seguras=total_seguras,
        moderadas=total_moderadas,
        peligrosas=total_peligrosas
    )

@app.route('/mapa')
def ver_mapa():
    """Sirve el mapa directamente"""
    try:
        with open('mapa_temp.html', 'r', encoding='utf-8') as f:
            return f.read()
    except:
        return "<html><body><h1>Generando mapa...</h1><script>setTimeout(() => location.reload(), 3000);</script></body></html>"

# =====================================================
# MAIN
# =====================================================
if __name__ == '__main__':
    print("=" * 70)
    print("🚀 SISTEMA DE MAPAS EN TIEMPO REAL - VERSIÓN ROBUSTA")
    print("=" * 70)
    print(f"📋 Fuente: Google Sheets")
    print(f"⏱️  Auto-actualización: cada {INTERVALO_REFRESH}s")
    print(f"🌐 Puerto: {PUERTO}")
    print("=" * 70)
    
    print("\n[...] Generando mapa inicial...")
    
    if generar_mapa():
        print("\n[✅] ¡Sistema listo!")
    else:
        print("\n[⚠️] Revisa los mensajes de arriba para ver qué falta")
    
    thread = threading.Thread(target=actualizar_periodicamente, daemon=True)
    thread.start()
    
    print(f"\n[🌐] Servidor: http://127.0.0.1:{PUERTO}")
    print("\n   Presiona Ctrl+C para detener\n")
    print("=" * 70)
    
    app.run(debug=False, host='0.0.0.0', port=PUERTO)

ModuleNotFoundError: No module named 'sklearn'